# Building Trees using scikit-learn - Lab

## Introduction

Following the simple example you saw in the previous lesson, you'll now build a decision tree for a more complex dataset. This lab covers all major areas of standard machine learning practice, from data acquisition to evaluation of results. We'll continue to use the Scikit-learn and Pandas libraries to conduct this analysis, following the same structure we saw in the previous lesson.

## Objectives

In this lab you will:

- Use scikit-learn to fit a decision tree classification model 
- Use entropy and information gain to identify the best attribute to split on at each node 
- Plot a decision tree using Python 

## UCI Banknote authentication dataset

In this lab, you'll work with a popular dataset for classification called the "UCI Bank note authentication dataset". This data was extracted from images that were taken from genuine and forged banknotes! The notes were first digitized, followed by a numerical transformation using DSP techniques. The final set of engineered features are all continuous in nature, meaning that our dataset consists entirely of floats, with no strings to worry about. If you're curious about how the dataset was created, you can visit the UCI link [here](https://archive.ics.uci.edu/ml/datasets/banknote+authentication)!

We have the following attributes in the dataset:  

1. __Variance__ of wavelet transformed image (continuous) 
2. __Skewness__ of wavelet transformed image (continuous) 
3. __Curtosis__ of wavelet transformed image (continuous) 
4. __Entropy__ of image (continuous) 
5. __Class__ (integer) - Target/Label 

## Step 1: Import the necessary libraries 

We've imported all the necessary modules you will require for this lab, go ahead and run the following cell: 

In [1]:
# Import necessary libraries
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier 
from sklearn.metrics import accuracy_score, roc_curve, auc
from sklearn.preprocessing import OneHotEncoder
from sklearn import tree

## Step 2: Import data

Now, you'll load our dataset in a DataFrame, perform some basic EDA, and get a general feel for the data you'll be working with.

- Import the file `'data_banknote_authentication.csv'` as a pandas DataFrame. Note that there is no header information in this dataset 
- Assign column names `'Variance'`, `'Skewness'`, `'Kurtosis'`, `'Entropy'`, and `'Class'` to the dataset in the given order 
- View the basic statistics and shape of the dataset 
- Check for the frequency of positive and negative examples in the target variable

In [2]:
# Create DataFrame

df = pd.read_csv('data_banknote_authentication.csv', header=None)


In [3]:
# Describe the dataset
df.columns = ['Variance', 'Skewness', 'Kurtosis', 'Entropy', 'Class']
df.info()


In [4]:
# Shape of dataset
print(df.head())


In [5]:
# Class frequency of target variable 
print("\n Class distribution:")
print(df['Class'].value_counts())


## Step 3: Create features, labels, training, and test data

Now we need to create our feature set `X` and labels `y`:  
- Create `X` and `y` by selecting the appropriate columns from the dataset
- Create a 80/20 split on the dataset for training/test. Use `random_state=10` for reproducibility

In [6]:
# Create features and labels

X = df.drop(columns=['Class'])
y = df['Class']


In [7]:
# Perform an 80/20 split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

print(f"\n Data Split: X_train: {X_train.shape}, X_test: {X_test.shape}, y_train: {y_train.shape}, y_test: {y_test.shape}")

## Step 4: Train the classifier and make predictions
- Create an instance of a decision tree classifier with `random_state=10` for reproducibility
- Fit the training data to the model 
- Use the trained model to make predictions with test data

In [8]:
# Train a DT classifier
clf = DecisionTreeClassifier(random_state=10)
# Train the classifier
clf.fit(X_train, y_train)


In [9]:
# Make predictions for test data
y_pred = clf.predict(X_test)

# Print predictions (first 10 values)
print("\n First 10 Predictions:", y_pred[:10])


## Step 5: Check predictive performance

Use different evaluation measures to check the predictive performance of the classifier: 
- Check the accuracy, AUC, and create a confusion matrix 
- Interpret the results 

In [10]:
# Calculate accuracy 
acc = accuracy_score(y_test, y_pred)
print(f"\n Accuracy Score: {acc:.4f}")  # Fixed variable name

# Check the AUC for predictions
false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_pred)  # Corrected function call
roc_auc = auc(false_positive_rate, true_positive_rate)
print('\nAUC is :{0}'.format(round(roc_auc, 2)))

# Create and print a confusion matrix 
print('\nConfusion Matrix')

## Level up (Optional)


### Re-grow the tree using entropy 

The default impurity criterion in scikit-learn is the Gini impurity. We can change it to entropy by passing in the argument `criterion='entropy'` to the classifier in the training phase.  

- Create an instance of a decision tree classifier with `random_state=10` for reproducibility. Make sure you use entropy to calculate impurity 
- Fit this classifier to the training data 
- Run the given code to plot the decision tree

In [11]:
# Import necessary libraries (fixing NameError)
from sklearn.metrics import accuracy_score, roc_curve, auc, confusion_matrix  #  Ensure confusion_matrix is imported

# Train a DecisionTreeClassifier using "entropy" as the impurity criterion
classifier_2 = DecisionTreeClassifier(criterion='entropy', random_state=10)

# Train the classifier (Fixing NotFittedError)
classifier_2.fit(X_train, y_train)  #  Ensure the model is trained before using it

# Make predictions on the test set
y_pred_entropy = classifier_2.predict(X_test)

#  Calculate Accuracy Score
accuracy_entropy = accuracy_score(y_test, y_pred_entropy)
print(' Accuracy Score: {:.4f}'.format(accuracy_entropy))

# Compute AUC Score
false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_pred_entropy)
roc_auc = auc(false_positive_rate, true_positive_rate)
print('\n AUC Score: {:.4f}'.format(roc_auc))

#  Compute Confusion Matrix (Fixing NameError)
conf_matrix_entropy = confusion_matrix(y_test, y_pred_entropy)

#  Print Confusion Matrix
print("\n Confusion Matrix:")
print(conf_matrix_entropy)


In [12]:
# Plot and show decision tree
plt.figure(figsize=(12,12), dpi=500)
tree.plot_tree(classifier_2, 
               feature_names=X.columns,
               class_names=np.unique(y).astype('str'),
               filled=True, rounded=True)
plt.show()

- We discussed earlier that decision trees are very sensitive to outliers. Try to identify and remove/fix any possible outliers in the dataset.

In [13]:
#  Detect outliers using boxplots
plt.figure(figsize=(10, 6))
df.boxplot()
plt.title("Outlier Detection - Boxplot of Features")
plt.xticks(rotation=45)
plt.show()

#  Define a function to remove outliers using the IQR method
def remove_outliers(df, columns):
    cleaned_df = df.copy()
    for col in columns:
        Q1 = cleaned_df[col].quantile(0.25)
        Q3 = cleaned_df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        cleaned_df = cleaned_df[(cleaned_df[col] >= lower_bound) & (cleaned_df[col] <= upper_bound)]
    return cleaned_df

#  Apply outlier removal to all numeric columns
df_cleaned = remove_outliers(df, ['Variance', 'Skewness', 'Kurtosis', 'Entropy'])

# Display the shape before and after outlier removal
print(f" Original dataset size: {df.shape[0]} rows")
print(f" Cleaned dataset size after outlier removal: {df_cleaned.shape[0]} rows")


- Check the distributions of the data. Is there any room for normalization/scaling of the data? Apply these techniques and see if it improves the accuracy score.

In [14]:
#  Plot histograms to visualize feature distributions
df_cleaned.hist(figsize=(10, 6), bins=20, color="blue", edgecolor="black")
plt.suptitle("Feature Distributions (After Outlier Removal)")
plt.show()

from sklearn.preprocessing import StandardScaler, MinMaxScaler

# Standardize Data Using StandardScaler
scaler = StandardScaler()
df_standardized = pd.DataFrame(scaler.fit_transform(df_cleaned.drop(columns=['Class'])), columns=df_cleaned.drop(columns=['Class']).columns)

#  Normalize Data Using MinMaxScaler
scaler_minmax = MinMaxScaler()
df_normalized = pd.DataFrame(scaler_minmax.fit_transform(df_cleaned.drop(columns=['Class'])), columns=df_cleaned.drop(columns=['Class']).columns)

#  Print Summary Statistics
print("\n Summary Statistics After Standardization:")
print(df_standardized.describe())

print("\n Summary Statistics After Normalization:")
print(df_normalized.describe())


## Summary 

In this lesson, we looked at growing a decision tree for the banknote authentication dataset, which is composed of extracted continuous features from photographic data. We looked at data acquisition, training, prediction, and evaluation. We also looked at growing trees using entropy vs. gini impurity criteria. In following lessons, we shall look at more pre-training tuning techniques for ensuring an optimal classifier for learning and prediction.  